# Homework 6

In [3]:
import pandas as pd
import numpy as np
from sympy import Symbol, lambdify
import pdb
import math

In [4]:
train_data = pd.read_csv('Input/training.dat', sep=' ', header=None, names=['x', 'y']);
test_data = pd.read_csv('Input/test.dat', sep=' ', header=None, names=['x', 'y']);

x_train = np.array(train_data['x'])
y_train = np.array(train_data['y'])

w0 = Symbol("w0")
w1 = Symbol("w1")
w2 = Symbol("w2")

func_a = np.sum(np.square(y_train - w0 - w1 * x_train))
f_a = lambdify([[w0, w1]], func_a, "numpy")
gf_a = lambdify([[w0, w1]], func_a.diff([[w0, w1]]), "numpy")
grad_fa = lambda x_arr : np.array(gf_a(x_arr), 'float64').reshape(1,len(x_arr))

func_b = np.sum(np.square(y_train - w0 - w1 * x_train - w2 * x_train**2))
f_b = lambdify([[w0, w1, w2]], func_b, "numpy")
gf_b = lambdify([[w0, w1, w2]], func_b.diff([[w0, w1, w2]]), "numpy")
grad_fb = lambda x_arr : np.array(gf_b(x_arr), 'float64').reshape(1,len(x_arr))

### Useful Functions

In [15]:
np_str = lambda x_k : np.array2string(x_k.reshape(len(x_k)), precision=3, separator=',')

f_str = lambda x : "{0:.4f}".format(x)

In [16]:
class OutputTable:    
    def __init__(self):
        self.table = pd.DataFrame([],columns=['k', 'x^k', 'f(x^k)', 'd^k', 'a^k', 'x^k+1'])
    def add_row(self, k, xk, fxk, dk, ak, xkp):
        self.table.loc[len(self.table)] = [k, np_str(xk), f_str(fxk.item()), np_str(dk), ak, np_str(xkp)]
    def print_latex(self):
        print(self.table.to_latex(index=False))

## Part A : Least Square Method with Steepest Descent

### Exact Line Search

In [17]:
def BisectionMethod(f,epsilon, a=-100,b=100) :
    iteration=0
    while (b - a) >= epsilon:
        x_1 = (a + b) / 2
        fx_1 = f(x_1)
        if f(x_1 + epsilon) <= fx_1:
            a = x_1
        else:
            b = x_1
        iteration+=1
    x_star = (a+b)/2
    return x_star

def ExactLineSearch(f, x0, d, eps=0.0000001):
    alpha = Symbol('alpha')
    function_alpha = f(np.array(x0)+alpha*np.array(d))
    f_alp = lambdify(alpha, function_alpha, 'numpy')
    alp_star = BisectionMethod(f_alp, epsilon=eps)
    return alp_star

### Steepest Descent Method

In [20]:
def steepestDescentMethod(f, grad_f, x_0, epsilon):
    xk = np.array(x_0).reshape(2,1)
    k = 0
    stop = False
    output = OutputTable()
    while(stop == False):
        d = - np.transpose(grad_f(xk))
        if(np.linalg.norm(d) < epsilon):
            stop = True
        else:
            a = ExactLineSearch(f,xk,d)
            xkp = xk + a*d
            output.add_row(k, xk, f(xk), d, a, xkp)
            k += 1
            xk = xkp
    output.add_row(k,xk,f(xk),d,None,np.array([]))
    return xk, np.asscalar(f(xk)), output

In [19]:
xk = np.array([0,0]).reshape(2,1)
xk

array([[0],
       [0]])

In [433]:
#ws_a, fs_a, outputs_a = steepestDescentMethod(f_a, grad_fa, [0,0], 0.001)
#ws_a, fs_a

In [ ]:
ws_b, fs_b, outputs_b = steepestDescentMethod(f_b, grad_fb, [0,0], 0.001)
ws_b, fs_b

## Part B : Nonlinear Regression:

In [5]:
x_train = np.array(train_data['x'])
y_train = np.array(train_data['y'])



In [71]:
def backpropagation(input_data,label_data,learning_rate=0.9,alpha=0.5,epsilon=0.001):
    epsilon=epsilon
    t=0
    x_train=input_data.reshape(100,1)
    y_train=label_data.reshape(100,1)
    all_data=np.full((100, 1), -1)
    x_train=np.concatenate((all_data,x_train),axis=1)
    weight_first=np.random.rand(3,2)
    weight_second=np.random.rand(1,4)#2 hidden units
    h=np.random.rand(100,3)
    H=np.random.rand(100,3)
    o=np.random.rand(100,1)
    O=np.random.rand(100,1)
    sigma_output=np.random.rand(100,1)
    sigma_hidden=np.random.rand(100,3)
    Error_value=0
    delta_big=np.random.rand(1,4)
    delta_small=np.random.rand(3,2)
    while(True): 
        np.random.shuffle(x_train)     
        for p in range(100):
            
            for j in range(3):
                sum_first=0
                for k in range(2):
                    
                    sum_first=sum_first+((weight_first[j,k]*x_train[p,k]))
                h[p,j]=sum_first
                H[p,j] = 1/(1+np.exp(-h[p,j]))
            
                
            all_data_2=np.full((100,1),-1)
            all_data_2=np.concatenate((all_data_2,H),axis=1)
            H_with_zero=all_data_2

            for i in range(1):
                sum_second=0
                for j in range(4):
                    sum_second=sum_second+weight_second[i,j]*H_with_zero[p,j]
                o[p,i]=sum_second
                O[p,i]=o[p,i]
            
            
                
            
            
            for i in range(1):
                sigma_output[p,i]=y_train[p,i]-O[p,i]
               
            for j in range(3): 
                sum_third=0
                for i in range(1):
                    sum_third=sum_third+(weight_second[i,j]*sigma_output[p,i])
               
                sigma_hidden[p,j]=H[p,j]*(1-H[p,j])*sum_third
                
            for j in range(4):    
                for i in range(1): 
                    delta_big[i,j]=learning_rate*sigma_output[p,i]*H_with_zero[p,j]
                    weight_second[i,j]=weight_second[i,j]+delta_big[i,j]
                    
            for k in range(2):   
                for j in range(3): 
                    
                    delta_small[j,k]=learning_rate*sigma_hidden[p,j]*x_train[p,k]
                    weight_first[j,k]=weight_first[j,k]+delta_small[j,k]
                    
        
        summation = 0  #variable to store the summation of differences
        n = len(y_train) #finding total number of items in list
        for i in range (0,n):  #looping through each element of the list
            difference = O[i] - y_train[i]  #finding the difference between observed and predicted value
            squared_difference = difference**2  #taking square of the differene 
            summation = summation + squared_difference  #taking a sum of all the differences
        MSE = summation/2  #dividing summation by 2
        print( "The Mean Square Error is: " , MSE)
        learning_rate=learning_rate*alpha
        t=t+1
        
        if learning_rate<epsilon:
            break
    return weight_first,weight_second,MSE


In [70]:
weight_1,weight_2,Error=backpropagation(x_train,y_train)

The Mean Square Error is:  [7.4494729e+52]
The Mean Square Error is:  [1.60450185e+53]
The Mean Square Error is:  [2508914.21113687]
The Mean Square Error is:  [2163073.07438283]
The Mean Square Error is:  [2447429.08263533]
The Mean Square Error is:  [2986844.92771541]
The Mean Square Error is:  [3764765.14688899]
The Mean Square Error is:  [4284833.63811607]
The Mean Square Error is:  [4284279.57421693]
The Mean Square Error is:  [4190867.17997067]
